In [1]:
import json
import os
import yaml
import random

from openai import AsyncOpenAI

from config import settings
from src.action_predictor import (
    CharcterActionPredictor
)
from src.character_store import CharacterStore

In [2]:
# Initialize OpenAI provider
print(settings.llm_model)
# OpenAI Client
client = AsyncOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


In [3]:
character_store = CharacterStore(
    character_dir="assets/characters",
    uids=[
        "35f0c56f-263d-42df-846c-e1833d8ca0ab"
    ]
)

with open("assets/sims_interactions.json", "r") as f:
    all_actions = json.load(f)
n = 10
actions = {k: random.sample(v, min(len(v), n)) for k,v in all_actions.items()}
print(actions.keys())

dict_keys(['Friendly', 'Ask', 'Romantic', 'Mean', 'Neutral'])


In [4]:
with open('prompts/action_predictor.yaml', 'r') as file:
    prompt = yaml.load(file, Loader=yaml.FullLoader)

In [5]:
predictor = CharcterActionPredictor(
    client=client,
    prompt=prompt,
    actions=actions,
    character_store=character_store,
    model="gpt-4.1-nano"
)

In [6]:
scene_context="Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism."

predicted_action = await predictor.forward(
    scene_context=scene_context,
    uid="35f0c56f-263d-42df-846c-e1833d8ca0ab",
    related_uids=[]
)

Information:
[Character]
description:
{"name":"Zephyr Orion","gender":"male","age":28,"dialogue_tone":"playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.","career":"Astronaut","personality_traits":[{"trait":"Goofball","description":"Enjoys joking and making others laugh, bringing a playful spirit to social situations."},{"trait":"Materialistic","description":"Loves acquiring new possessions and often leans towards bragging about them."},{"trait":"Outgoing","description":"Flourishes in social situations and enjoys being around people."},{"trait":"Gloomy","description":"Grows sad when left alone for too long."},{"trait":"Ambitious","description":"Continuously strives to reach new milestones in his career."}],"hobbies":["Fitness","Cooking","Painting"],"living_conditions":["Resides in a modern city apartment filled with space memorabilia and art supplies.","Leads a healthy lifesty

In [8]:
print(json.dumps(predicted_action, indent=2))

{
  "action": "Zephyr smirks mischievously and shifts his tone to a more playful one, saying, 'You know, I once had a space fashion show on the moon\u2014absolutely out of this world!'",
  "targets": []
}
